#### Goal
Get images linked back to their titles

In [18]:
import sys
import os
import pandas as pd

# Data storage
from sqlalchemy import create_engine # SQL helper
import psycopg2 as psql #PostgreSQL DBs

sys.path.append("..")

In [89]:
import keys
import data_fcns as dfc

Images directory:

I set up an s3 bucket: https://comrx.s3-us-west-2.amazonaws.com/covers/

In [10]:
# Define path to secret
secret_path_aws = os.path.join(os.environ['HOME'], '.secret', 
                           'aws_ps_flatiron.json')
secret_path_aws

'/Users/werlindo/.secret/aws_ps_flatiron.json'

In [19]:
# Define path to secret
secret_path_aws = os.path.join(os.environ['HOME'], '.secret', 
                           'aws_ps_flatiron.json')
secret_path_aws

aws_keys = keys.get_keys(secret_path_aws)
user = aws_keys['user']
ps = aws_keys['password']
host = aws_keys['host']
db = aws_keys['db_name']

aws_ps_engine = ('postgresql://' + user + ':' + ps + '@' + host + '/' + db)

# Setup PSQL connection
conn = psql.connect(
    database=db,
    user=user,
    password=ps,
    host=host,
    port='5432'
)

In [20]:
# Instantiate cursor
cur = conn.cursor()

In [53]:
#  Count records.
query = """
    SELECT * from comics LIMIT 100;
"""

In [50]:
#  Count records.
query_alter = """
    ALTER TABLE comics 
    ADD COLUMN img_url VARCHAR;
"""

In [30]:
tables = """
SELECT table_name
  FROM information_schema.tables
 WHERE table_schema='public'
   AND table_type='BASE TABLE';
   """

ALTER TABLE table_name
ADD COLUMN new_column_name data_type;

In [51]:
# Execute the query
cur.execute(query_alter)

In [54]:
# Execute the query
cur.execute(query)

In [55]:
conn.commit()

In [56]:
# Check results
temp_df = pd.DataFrame(cur.fetchall())
temp_df.columns = [col.name for col in cur.description]

In [69]:
temp_df.head()

,comic_id,comic_title,img_url
0,1,0Secret Wars (Marvel),None
1,2,100 Bullets Brother Lono (DC),None
2,3,100 Penny Press Locke & Key (IDW),None
3,4,100 Penny Press Star Trek (IDW),None
4,5,100 Penny Press Thunder Agent (IDW),None


Cool. Let's practice updating.

In [74]:
query_update = """
UPDATE comics SET img_url= 'https://comrx.s3-us-west-2.amazonaws.com/covers/wonder_woman.jpg'
WHERE comic_title = 'Wonder Woman (DC)';
"""

In [80]:
#  Count records.
query = """
    SELECT * from comics WHERE comic_title = 'Wonder Woman (DC)';
"""

In [81]:
# Execute the query
cur.execute(query)

In [77]:
# Execute the query
cur.execute(query_update)

In [76]:
conn.rollback()

In [78]:
conn.commit()

In [82]:
# Check results
temp_df = pd.DataFrame(cur.fetchall())
temp_df.columns = [col.name for col in cur.description]

In [83]:
temp_df.head()

,comic_id,comic_title,img_url
0,7021,Wonder Woman (DC),https://comrx.s3-us-west-2.amazonaws.com/cover...


Yay.

In [58]:
!ls

__init__.py
archive
assets
comic_recs_part.py
comics_rx-1_data_prep.ipynb
comics_rx-2_eda.ipynb
comics_rx-3_als_all_data.ipynb
comics_rx-4_als_reduced_data.ipynb
comics_rx-5_pseudo_deploy.ipynb
comics_rx-6_als_reduced_data_gs_cv.ipynb
comics_rx-7_mvp_dev.ipynb
comics_rx-8_recommender_poc.ipynb
comics_rx-9_scrape_comic_covers.ipynb
comics_rx-9_scrape_comic_covers_win.ipynb
drivers
get_images.ipynb
models
raw_data
references
scratch
support_data
versions


We need a crosswalk from `comic_title` to `img_url`. For that we need to rebuild the `title` I've been using to search on images, because that's what I've been using to label the jpegs.

In [84]:
#  Count records.
query = """
    SELECT * from comic_trans;
"""

In [108]:
# Execute the query
cur.execute(query)

In [109]:
# Check results
temp_df = pd.DataFrame(cur.fetchall())
temp_df.columns = [col.name for col in cur.description]

In [110]:
temp_df.head(3)

,index,publisher,item_id,title_and_num,qty_sold,date_sold,account_num,comic_title
0,2,Amaze Ink Slave Labor Graphics,DCD151935,Filler Bunny #2,1,2011-08-14 18:01:03,00174,Filler Bunny (SLG)
1,3,Amaze Ink Slave Labor Graphics,DCD341726,Gargoyles #6,1,2012-06-22 14:11:37,00593,Gargoyles (SLG)
2,4,Amaze Ink Slave Labor Graphics,DCD416182,Royal Historian of Oz #1,1,2010-07-21 14:03:07,00226,Royal Historian of Oz (SLG)


In [111]:
temp_df['title'] = ( temp_df['title_and_num'].apply(dfc.cut_issue_num) )

In [112]:
temp_df.head()

,index,publisher,item_id,title_and_num,qty_sold,date_sold,account_num,comic_title,title
0,2,Amaze Ink Slave Labor Graphics,DCD151935,Filler Bunny #2,1,2011-08-14 18:01:03,00174,Filler Bunny (SLG),Filler Bunny
1,3,Amaze Ink Slave Labor Graphics,DCD341726,Gargoyles #6,1,2012-06-22 14:11:37,00593,Gargoyles (SLG),Gargoyles
2,4,Amaze Ink Slave Labor Graphics,DCD416182,Royal Historian of Oz #1,1,2010-07-21 14:03:07,00226,Royal Historian of Oz (SLG),Royal Historian of Oz
3,5,Amaze Ink Slave Labor Graphics,DCD416182,Royal Historian of Oz #1,1,2010-07-14 19:49:40,00399,Royal Historian of Oz (SLG),Royal Historian of Oz
4,6,Amaze Ink Slave Labor Graphics,DCD416182,Royal Historian of Oz #1,1,2010-07-19 10:39:04,00237,Royal Historian of Oz (SLG),Royal Historian of Oz


In [113]:
temp_df['title'] = (temp_df['title'].apply(lambda x : x.replace('&' ,'and'))
                    .apply(lambda x : x.replace('?' ,''))
                    .apply(lambda x : x.replace('/' ,' '))
                   )

In [114]:
temp_df.head()

,index,publisher,item_id,title_and_num,qty_sold,date_sold,account_num,comic_title,title
0,2,Amaze Ink Slave Labor Graphics,DCD151935,Filler Bunny #2,1,2011-08-14 18:01:03,00174,Filler Bunny (SLG),Filler Bunny
1,3,Amaze Ink Slave Labor Graphics,DCD341726,Gargoyles #6,1,2012-06-22 14:11:37,00593,Gargoyles (SLG),Gargoyles
2,4,Amaze Ink Slave Labor Graphics,DCD416182,Royal Historian of Oz #1,1,2010-07-21 14:03:07,00226,Royal Historian of Oz (SLG),Royal Historian of Oz
3,5,Amaze Ink Slave Labor Graphics,DCD416182,Royal Historian of Oz #1,1,2010-07-14 19:49:40,00399,Royal Historian of Oz (SLG),Royal Historian of Oz
4,6,Amaze Ink Slave Labor Graphics,DCD416182,Royal Historian of Oz #1,1,2010-07-19 10:39:04,00237,Royal Historian of Oz (SLG),Royal Historian of Oz


In [115]:
work = temp_df.groupby(['comic_title', 'title'], as_index=False).index.count()

In [116]:
work.head()

,comic_title,title,index
0,0Secret Wars (Marvel),0Secret Wars,1
1,1 For $1 Abe Sapien (Dark Horse),1 For $1 Abe Sapien,3
2,1 For $1 Action Philosopher (Dark Horse),1 For $1 Action Philosopher,3
3,1 For $1 Avatar Last Airbende (Dark Horse),1 For $1 Avatar Last Airbende,2
4,1 For $1 Axe Cop Bad Guy Eart (Dark Horse),1 For $1 Axe Cop Bad Guy Eart,1


In [117]:
work.shape

(7202, 3)

In [150]:
work['filename'] = work['title'].apply(lambda x: x.lower().replace(' ', '_')) + '.jpg'

In [134]:
search_path = 'https://comrx.s3-us-west-2.amazonaws.com/covers/'
search_path

'https://comrx.s3-us-west-2.amazonaws.com/covers/'

In [151]:
work['search_path'] = search_path + work['filename']

In [153]:
work.loc[work['title']=='Wonder Woman']['search_path']

7011    https://comrx.s3-us-west-2.amazonaws.com/cover...
7012    https://comrx.s3-us-west-2.amazonaws.com/cover...
Name: search_path, dtype: object

In [162]:
work.loc[work['comic_title']=='Wonder Woman (DC)']['filename'].values[0]

'wonder_woman.jpg'

In [152]:
work.head()

,comic_title,title,index,search_path,filename
0,0Secret Wars (Marvel),0Secret Wars,1,https://comrx.s3-us-west-2.amazonaws.com/cover...,0secret_wars.jpg
1,1 For $1 Abe Sapien (Dark Horse),1 For $1 Abe Sapien,3,https://comrx.s3-us-west-2.amazonaws.com/cover...,1_for_$1_abe_sapien.jpg
2,1 For $1 Action Philosopher (Dark Horse),1 For $1 Action Philosopher,3,https://comrx.s3-us-west-2.amazonaws.com/cover...,1_for_$1_action_philosopher.jpg
3,1 For $1 Avatar Last Airbende (Dark Horse),1 For $1 Avatar Last Airbende,2,https://comrx.s3-us-west-2.amazonaws.com/cover...,1_for_$1_avatar_last_airbende.jpg
4,1 For $1 Axe Cop Bad Guy Eart (Dark Horse),1 For $1 Axe Cop Bad Guy Eart,1,https://comrx.s3-us-west-2.amazonaws.com/cover...,1_for_$1_axe_cop_bad_guy_eart.jpg


In [202]:
tests = ['Wonder Woman (DC)', 'Batman (DC)', 'X-Men (Marvel)', 'Werlindo']

In [60]:
client = boto3.client('s3')

In [178]:
ttl = 'wonder woman (dc)'
url = 'www.overhere.com'

In [188]:
ssql = ("SELECT * FROM '" + ttl + "';" + 
       "WHERE huh ")

In [189]:
ssql

"SELECT * FROM 'wonder woman (dc)';WHERE huh "

In [203]:
for test in tests:
    # Find file name
    try:
        got_filename = work.loc[work['comic_title']==test]['filename'].values[0]
        client.head_object(Bucket='comrx', Key='covers/' + got_filename)
        print('got ' + got_filename)
        
        # Update comics table on AWS
        img_url = work.loc[work['comic_title']==test]['search_path'].values[0]
        print(img_url)
        
        # Build query string
        query_update = ("UPDATE comics " + 
                        "SET img_url = '" + img_url + "' " +
                        "WHERE comic_title = '" + test +
                        "';"
                       )
        print(query_update)
        
        # Execute query
        cur.execute(query_update)
        conn.commit()
           
    except:
        pass
        print('wut')
#     print(test)
#     print(work.loc[work['comic_title']==test]['filename'].values[0])
    
#         client.head_object(Bucket='comrx', Key='covers/wonder_woman.jpg')

got wonder_woman.jpg
https://comrx.s3-us-west-2.amazonaws.com/covers/wonder_woman.jpg
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/wonder_woman.jpg' WHERE comic_title = 'Wonder Woman (DC)';
got batman.jpg
https://comrx.s3-us-west-2.amazonaws.com/covers/batman.jpg
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/batman.jpg' WHERE comic_title = 'Batman (DC)';
wut
wut


In [194]:
#  Count records.
query = """
    SELECT * from comics WHERE comic_title = 'Wonder Woman (DC)';
"""

In [204]:
#  Count records.
query = """
    SELECT * from comics WHERE comic_title = 'Batman (DC)';
"""

In [205]:
# Execute the query
cur.execute(query)

In [206]:
# Check results
temp_df = pd.DataFrame(cur.fetchall())
temp_df.columns = [col.name for col in cur.description]

In [207]:
temp_df.head()

,comic_id,comic_title,img_url
0,610,Batman (DC),https://comrx.s3-us-west-2.amazonaws.com/cover...


In [65]:
ww = temp_df.loc[temp_df['comic_title']=='wonder_woman'].copy()

In [66]:
ww

,comic_id,comic_title,img_url


In [59]:
import boto3

In [60]:
client = boto3.client('s3')

In [63]:
client.head_object(Bucket='comrx', Key='covers/wonder_woman.jpg')

{'ResponseMetadata': {'RequestId': '9C4E154037F911A3',
  'HostId': '7KFSE0MMYxxUCk5zx6jK0/erM7D7R/MRiML48f33/dmZ5l68BeTyvGdnalEyHbNYmb4JJDVTMNM=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '7KFSE0MMYxxUCk5zx6jK0/erM7D7R/MRiML48f33/dmZ5l68BeTyvGdnalEyHbNYmb4JJDVTMNM=',
   'x-amz-request-id': '9C4E154037F911A3',
   'date': 'Sat, 06 Jul 2019 18:46:55 GMT',
   'last-modified': 'Sat, 06 Jul 2019 18:17:08 GMT',
   'etag': '"22586bbc17033fe936f1f09a49687039"',
   'x-amz-version-id': 'Edfhk0i8IWNF5.ueHTy3Taw6rnozfnWe',
   'accept-ranges': 'bytes',
   'content-type': 'image/jpeg',
   'content-length': '19300',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'AcceptRanges': 'bytes',
 'LastModified': datetime.datetime(2019, 7, 6, 18, 17, 8, tzinfo=tzutc()),
 'ContentLength': 19300,
 'ETag': '"22586bbc17033fe936f1f09a49687039"',
 'VersionId': 'Edfhk0i8IWNF5.ueHTy3Taw6rnozfnWe',
 'ContentType': 'image/jpeg',
 'Metadata': {}}

In [ ]:
# enumerate local files recursively
for root, dirs, files in os.walk(local_directory):

    for filename in files:

    # construct the full local path
    #local_path = os.path.join(root, filename)

    # construct the full Dropbox path
    #relative_path = os.path.relpath(local_path, local_directory)
#    s3_path = os.path.join(destination, relative_path)
    s3_path = os.path.join(destination, filename)

    # relative_path = os.path.relpath(os.path.join(root, filename))

    print 'Searching "%s" in "%s"' % (s3_path, bucket)
    try:
        client.head_object(Bucket=bucket, Key=s3_path)